# Similarity Pipeline Demo

This notebook demonstrates how to assemble the similarity pipeline components shipped with `spark-fuse` to cluster semantically similar rows and select per-cluster representatives.

## 1. Start a local Spark session

For demonstration purposes the session runs in local mode. In production you would rely on your existing cluster settings.

In [1]:
import os, sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [2]:
from spark_fuse.spark import create_session

spark = create_session(app_name="spark-fuse-similarity-demo", master="local[2]")

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kevin/.ivy2/cache
The jars for the packages stored in: /Users/kevin/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-946a2238-5023-4616-84ab-cac775a8d6ba;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 326ms :: artifacts dl 31ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

## 2. Create a sample dataset

Each row carries a simple three-dimensional embedding and a quality score that we can later use to pick representatives.

In [3]:
data = [
    (1, "Crunchy Red Apple", 4.7),
    (2, "Sweet Gala Apple", 4.9),
    (3, "Fresh Cavendish Banana", 4.6),
    (4, "Ripe Plantain", 4.5),
    (5, "Classic Spiral Notebook", 4.4),
]
columns = ["product_id", "description", "score"]
df = spark.createDataFrame(data, columns)

df.show(truncate=False)


+----------+-----------------------+-----+
|product_id|description            |score|
+----------+-----------------------+-----+
|1         |Crunchy Red Apple      |4.7  |
|2         |Sweet Gala Apple       |4.9  |
|3         |Fresh Cavendish Banana |4.6  |
|4         |Ripe Plantain          |4.5  |
|5         |Classic Spiral Notebook|4.4  |
+----------+-----------------------+-----+



## 3. Configure and run the similarity pipeline

The pipeline embeds the product descriptions with Hugging Face `sentence-transformers`, normalizes vectors for cosine similarity, clusters with K-Means, and keeps the highest scoring item per cluster. If the dependency is missing in your environment, the generator falls back to a deterministic stub so the demo continues to run.


In [4]:
from spark_fuse.similarity import (
    CosineSimilarity,
    KMeansPartitioner,
    MaxColumnChoice,
    SentenceEmbeddingGenerator,
    SimilarityPipeline,
)

embedding_generator = SentenceEmbeddingGenerator(
    input_col="description",
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    normalize=True,
    use_vectorized=False,
)
metric = CosineSimilarity(embedding_col="embedding")
partitioner = KMeansPartitioner(k=3, seed=7)
choice = MaxColumnChoice(column="score")

pipeline = SimilarityPipeline(
    embedding_generator=embedding_generator,
    partitioner=partitioner,
    similarity_metric=metric,
    choice_function=choice,
)

clustered_df = pipeline.run(df)
clustered_df.select("product_id", "cluster_id", "description").orderBy("cluster_id").show(truncate=False)


25/10/29 14:44:48 ERROR Executor: Exception in task 1.0 in stage 2.0 (TID 3)/ 2]
org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.It

Py4JJavaError: An error occurred while calling o95.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 3) (air-von-kevin executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterato

## 4. Retrieve representatives

With `MaxColumnChoice` the representative is simply the row with the largest score inside each cluster.

In [ ]:
representatives = pipeline.select_representatives(clustered_df)
representatives.select("cluster_id", "product_id", "description", "score").orderBy("cluster_id").show(truncate=False)

## 5. Shut down Spark

In [ ]:
spark.stop()